In [1]:
import os
from collections import defaultdict
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding,LSTM, Dense, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import binary_crossentropy
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from tensorflow.keras.models import load_model
import lizard
import json
from tqdm import tqdm

In [2]:
cyclomatic_complex = lizard.analyze_file('../code_forces/1_A/300963.txt')
print(cyclomatic_complex.__dict__)
for i in cyclomatic_complex.function_list:
    print(i.__dict__)

{'filename': '../code_forces/1_A/300963.txt', 'nloc': 2, 'function_list': [<lizard.FunctionInfo object at 0x113bfc1c0>], 'token_count': 61}
{'cyclomatic_complexity': 1, 'nloc': 1, 'token_count': 52, 'name': 'main', 'long_name': 'main()', 'start_line': 2, 'end_line': 2, 'full_parameters': [], 'filename': '../code_forces/1_A/300963.txt', 'top_nesting_level': 0, 'fan_in': 0, 'fan_out': 0, 'general_fan_out': 0}


In [3]:
summary = list()
with open('../problem_set.json','r') as file:
    problem_info = json.load(file)
problems = list()
with open('../_tags.txt') as file:
    content = file.readlines()
    for line in content:
        problem = list(eval(line).items())[0][0]
        problems.append(problem)
for problem in tqdm(problems):
    if (problem != '.DS_Store'):
        for code in os.listdir('../code_forces/' + problem + '/'):
            if (code != '.DS_Store'):
                [contest_num, problem_index] = problem.split('_')
                for index in problem_info['result']['problems']:
                    if (index['contestId'] == int(contest_num)) and (index['index'] == problem_index):
                        info_list = {}
                        info_list['problem'] = problem
                        info_list['code'] = code
                        file_analyze = lizard.analyze_file('../code_forces/' + problem + '/' + code)
                        cc_num = 0
                        nloc = 0
                        token_count = 0
                        for i in file_analyze.function_list:
                            cc_num+=i.__dict__['cyclomatic_complexity']
                            nloc+=i.__dict__['nloc']
                            token_count+=i.__dict__['token_count']
                        info_list['ccn'] = cc_num
                        info_list['nloc'] = nloc
                        info_list['token_count'] = token_count
                        try:
                            info_list['rating'] = index['rating']
                        except: info_list['rating'] = 0
                summary.append(info_list)

100%|██████████| 2167/2167 [04:52<00:00,  7.40it/s]


In [5]:
with open('_metrics.txt','w') as file:
    for index in summary:
        file.write(str(index) + '\n')